# Colab setup

In [ ]:
%%capture
!pip install accelerate sentencepiece bitsandbytes huggingface-hub datasets sentence-transformers faiss-cpu

# General setup

In [1]:
import transformers
import faiss
import sentence_transformers
import torch
import re
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# LLama2

In [ ]:
checkpoint = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint,torch_dtype=torch.float16,device_map="auto")

# Load data

In [ ]:
hf_dset = load_dataset("liar")

train_df = hf_dset["train"].rename_column("statement", "text").to_pandas()
val_df = hf_dset["validation"].rename_column("statement", "text").to_pandas()
test_df = hf_dset["test"].rename_column("statement", "text").to_pandas()

train_df["id"] = train_df["id"].apply(lambda x: int(x.split(".")[0]))
val_df["id"] = val_df["id"].apply(lambda x: int(x.split(".")[0]))
test_df["id"] = test_df["id"].apply(lambda x: int(x.split(".")[0]))

train_df.label = train_df.label.apply(lambda x: 0 if x == 3 else 1)
val_df.label = val_df.label.apply(lambda x: 0 if x == 3 else 1)
test_df.label = test_df.label.apply(lambda x: 0 if x == 3 else 1)

# Evaluation

## Zero-shot prompting

In [ ]:
valid = 0
correct = 0
tp = 0
fp = 0
fn = 0
system_prompt = """
You are an expert in fake news. You have to follow the provided instructions precisely and answer only with true or false.

Here are some rules you always follow:

- Give the requested answer even if you don't have enough evidence. Use your internal knowledge.
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

for idx in tqdm(range(len(test_df))):
    txt = test_df.iloc[idx]["text"]
    label = test_df.iloc[idx]["label"]
    prompt = f"""
    To the best of your knowledge, you must choose if the input text is true or false, even if you don't have enough evidence.

    Input text:
    {txt}
    Response:
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt").to("cuda")
    outputs = model.generate(tokenized_chat, max_new_tokens=400)
    result = tokenizer.decode(outputs[0][tokenized_chat.shape[1]:], skip_special_tokens=True)
    if re.match("(False|True)", result.strip()) is not None:
        valid += 1
        pred = 1 if "False" in result else 0
        if pred == label and label == 1:
            tp += 1
        if pred == 0 and label == 1:
            fn += 1
        if pred == 1 and label == 0:
            fp += 1
        if pred == label:
            correct += 1

100%|██████████| 1283/1283 [39:06<00:00,  1.83s/it]


In [ ]:
print(f"{correct}/{valid}, Accuracy:{correct*100/valid:.2f}")

661/1085, Accuracy:60.92


In [ ]:
print(f"{correct}/{valid}, Accuracy:{correct*100/valid:.2f}")

682/1086, Accuracy:62.80


In [ ]:
print(f"Precision: {tp/(tp+fp):.3f}")
print(f"Recall: {tp/(tp+fn):.3f}")

Precision: 0.857
Recall: 0.657


## Few-shot prompting

In [ ]:
embedder = sentence_transformers.SentenceTransformer("all-MiniLM-L6-v2")
sentences = train_df["text"].tolist()
embeds = embedder.encode(sentences)

In [ ]:
d = embeds.shape[1]
k = 3
index = faiss.IndexFlatL2(d)
index.add(embeds)

In [ ]:
system_prompt = """
You are an expert in fake news. You have to follow the provided instructions precisely and answer only with true or false.
It is critical to answer True or False, even if there is not enough evidence. The world leaders need the precise answer.


Here are some rules you always follow:

- Give the requested answer even if you don't have enough evidence. Use your internal knowledge.
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

In [ ]:
valid = 0
correct = 0
tp = 0
fp = 0
fn = 0

for idx in tqdm(range(len(test_df))):
    txt = test_df.iloc[idx]["text"]
    label = test_df.iloc[idx]["label"]
    x_query = embedder.encode([txt])

    D, I = index.search(x_query, k)  # search
    examples = []
    I = I[0]
    for ex_idx, elem in enumerate(I):
      example_label = "False" if train_df.iloc[elem]["label"] != 3 else "True"
      examples.append("\n".join([f"Example {ex_idx+1}", f"Text: {train_df.iloc[elem]['text']}", f"Label: {example_label}"]))
    examples = "\n\n".join(examples)
    prompt = f"""
{examples}

Following the same format above from the examples, what is the label of the following text? You must choose True or False, even if you don't have enough evidence:

Text: {txt}
Label:
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
    ]
    tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt").to("cuda")
    outputs = model.generate(tokenized_chat, max_new_tokens=50)
    result = tokenizer.decode(outputs[0][tokenized_chat.shape[1]:], skip_special_tokens=True)
    if re.search("Label: ?(False|True)", result.strip()) is not None:
        valid += 1
        pred = 1 if "False" in result else 0
        if pred == label and label == 1:
            tp += 1
        if pred == 0 and label == 1:
            fn += 1
        if pred == 1 and label == 0:
            fp += 1
        if pred == label:
            correct += 1

100%|██████████| 1283/1283 [26:11<00:00,  1.22s/it]


In [ ]:
print(f"{correct}/{valid}, Accuracy:{correct*100/valid:.2f}")

322/843, Accuracy:38.20


In [ ]:
print(f"Precision: {tp/(tp+fp):.3f}")
print(f"Recall: {tp/(tp+fn):.3f}")

Precision: 0.804
Recall: 0.318
